# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f829c7b2eb0>
├── 'a' --> tensor([[ 0.6032, -1.1886,  0.0582],
│                   [-0.2652, -0.7569,  1.0518]])
└── 'x' --> <FastTreeValue 0x7f829c7b2df0>
    └── 'c' --> tensor([[-1.3675, -0.3173, -0.4794, -1.2517],
                        [-0.6807,  1.7353, -0.9463,  0.0941],
                        [-0.4180, -0.3393,  0.2552, -0.0229]])

In [4]:
t.a

tensor([[ 0.6032, -1.1886,  0.0582],
        [-0.2652, -0.7569,  1.0518]])

In [5]:
%timeit t.a

67.6 ns ± 0.0658 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f829c7b2eb0>
├── 'a' --> tensor([[ 0.4074, -0.0072, -0.9359],
│                   [ 0.5603,  0.1376, -0.9433]])
└── 'x' --> <FastTreeValue 0x7f829c7b2df0>
    └── 'c' --> tensor([[-1.3675, -0.3173, -0.4794, -1.2517],
                        [-0.6807,  1.7353, -0.9463,  0.0941],
                        [-0.4180, -0.3393,  0.2552, -0.0229]])

In [7]:
%timeit t.a = new_value

70.5 ns ± 0.05 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6032, -1.1886,  0.0582],
               [-0.2652, -0.7569,  1.0518]]),
    x: Batch(
           c: tensor([[-1.3675, -0.3173, -0.4794, -1.2517],
                      [-0.6807,  1.7353, -0.9463,  0.0941],
                      [-0.4180, -0.3393,  0.2552, -0.0229]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6032, -1.1886,  0.0582],
        [-0.2652, -0.7569,  1.0518]])

In [11]:
%timeit b.a

58.8 ns ± 0.101 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0523, -1.1465, -0.4067],
               [ 1.1525,  1.0854,  0.2439]]),
    x: Batch(
           c: tensor([[-1.3675, -0.3173, -0.4794, -1.2517],
                      [-0.6807,  1.7353, -0.9463,  0.0941],
                      [-0.4180, -0.3393,  0.2552, -0.0229]]),
       ),
)

In [13]:
%timeit b.a = new_value

506 ns ± 0.214 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

800 ns ± 0.0946 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 14.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

141 µs ± 731 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f81e29ef790>
├── 'a' --> tensor([[[ 0.6032, -1.1886,  0.0582],
│                    [-0.2652, -0.7569,  1.0518]],
│           
│                   [[ 0.6032, -1.1886,  0.0582],
│                    [-0.2652, -0.7569,  1.0518]],
│           
│                   [[ 0.6032, -1.1886,  0.0582],
│                    [-0.2652, -0.7569,  1.0518]],
│           
│                   [[ 0.6032, -1.1886,  0.0582],
│                    [-0.2652, -0.7569,  1.0518]],
│           
│                   [[ 0.6032, -1.1886,  0.0582],
│                    [-0.2652, -0.7569,  1.0518]],
│           
│                   [[ 0.6032, -1.1886,  0.0582],
│                    [-0.2652, -0.7569,  1.0518]],
│           
│                   [[ 0.6032, -1.1886,  0.0582],
│                    [-0.2652, -0.7569,  1.0518]],
│           
│                   [[ 0.6032, -1.1886,  0.0582],
│                    [-0.2652, -0.7569,  1.0518]]])
└── 'x' --> <FastTreeValue 0x7f8298730c10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 119 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f81e9654fd0>
├── 'a' --> tensor([[ 0.6032, -1.1886,  0.0582],
│                   [-0.2652, -0.7569,  1.0518],
│                   [ 0.6032, -1.1886,  0.0582],
│                   [-0.2652, -0.7569,  1.0518],
│                   [ 0.6032, -1.1886,  0.0582],
│                   [-0.2652, -0.7569,  1.0518],
│                   [ 0.6032, -1.1886,  0.0582],
│                   [-0.2652, -0.7569,  1.0518],
│                   [ 0.6032, -1.1886,  0.0582],
│                   [-0.2652, -0.7569,  1.0518],
│                   [ 0.6032, -1.1886,  0.0582],
│                   [-0.2652, -0.7569,  1.0518],
│                   [ 0.6032, -1.1886,  0.0582],
│                   [-0.2652, -0.7569,  1.0518],
│                   [ 0.6032, -1.1886,  0.0582],
│                   [-0.2652, -0.7569,  1.0518]])
└── 'x' --> <FastTreeValue 0x7f81e9654b80>
    └── 'c' --> tensor([[-1.3675, -0.3173, -0.4794, -1.2517],
                        [-0.6807,  1.7353, -0.9463,  0.0941],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 60.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

55.9 µs ± 59.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.3675, -0.3173, -0.4794, -1.2517],
                       [-0.6807,  1.7353, -0.9463,  0.0941],
                       [-0.4180, -0.3393,  0.2552, -0.0229]],
              
                      [[-1.3675, -0.3173, -0.4794, -1.2517],
                       [-0.6807,  1.7353, -0.9463,  0.0941],
                       [-0.4180, -0.3393,  0.2552, -0.0229]],
              
                      [[-1.3675, -0.3173, -0.4794, -1.2517],
                       [-0.6807,  1.7353, -0.9463,  0.0941],
                       [-0.4180, -0.3393,  0.2552, -0.0229]],
              
                      [[-1.3675, -0.3173, -0.4794, -1.2517],
                       [-0.6807,  1.7353, -0.9463,  0.0941],
                       [-0.4180, -0.3393,  0.2552, -0.0229]],
              
                      [[-1.3675, -0.3173, -0.4794, -1.2517],
                       [-0.6807,  1.7353, -0.9463,  0.0941],
                       [-0.4180, -0.3393,  0.2552, -0.0229]],

In [26]:
%timeit Batch.stack(batches)

78.5 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.3675, -0.3173, -0.4794, -1.2517],
                      [-0.6807,  1.7353, -0.9463,  0.0941],
                      [-0.4180, -0.3393,  0.2552, -0.0229],
                      [-1.3675, -0.3173, -0.4794, -1.2517],
                      [-0.6807,  1.7353, -0.9463,  0.0941],
                      [-0.4180, -0.3393,  0.2552, -0.0229],
                      [-1.3675, -0.3173, -0.4794, -1.2517],
                      [-0.6807,  1.7353, -0.9463,  0.0941],
                      [-0.4180, -0.3393,  0.2552, -0.0229],
                      [-1.3675, -0.3173, -0.4794, -1.2517],
                      [-0.6807,  1.7353, -0.9463,  0.0941],
                      [-0.4180, -0.3393,  0.2552, -0.0229],
                      [-1.3675, -0.3173, -0.4794, -1.2517],
                      [-0.6807,  1.7353, -0.9463,  0.0941],
                      [-0.4180, -0.3393,  0.2552, -0.0229],
                      [-1.3675, -0.3173, -0.4794, -1.2517],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 289 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

329 µs ± 752 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
